In [1]:
from __future__ import absolute_import, division, print_function
import scipy.io.wavfile as wav
import os, sys
import tensorflow as tf
import tensorflow.contrib.eager as tfe
import numpy as np
tf.enable_eager_execution()

In [2]:
tf.executing_eagerly()
wavs_path = "/home/neo/MS/TIMIT/TRAIN/DR1/FCJF0/"

In [3]:
def get_files():
    files = os.listdir(wavs_path)
    files = [ele for ele in files if "WAV" in ele]
    return files

In [4]:
files = get_files()
files.sort()

In [5]:
(rate,signals) = wav.read(wavs_path+"SA1.wav")
signals = signals.astype(np.float32)

In [6]:
frames = tf.contrib.signal.frame(signals, frame_length=1, frame_step=1)

In [7]:
magnitude_spectrograms = tf.abs(tf.contrib.signal.stft(
    signals, frame_length=1, frame_step=1, fft_length=256))

In [8]:
spectrogram_patches = tf.contrib.signal.frame(
    magnitude_spectrograms, frame_length=1, frame_step=1, axis=1)

In [9]:
num_spectrogram_bins = magnitude_spectrograms.shape[-1].value
lower_edge_hertz, upper_edge_hertz, num_mel_bins = 80.0, 7600.0, 64
linear_to_mel_weight_matrix = tf.contrib.signal.linear_to_mel_weight_matrix(
  num_mel_bins, num_spectrogram_bins, rate, lower_edge_hertz,
  upper_edge_hertz)
mel_spectrograms = tf.tensordot(
  magnitude_spectrograms, linear_to_mel_weight_matrix, 1)
# Note: Shape inference for `tf.tensordot` does not currently handle this case.
mel_spectrograms.set_shape(magnitude_spectrograms.shape[:-1].concatenate(
  linear_to_mel_weight_matrix.shape[-1:]))

In [10]:
log_offset = 1e-6
log_mel_spectrograms = tf.log(mel_spectrograms + log_offset)

In [11]:
num_mfccs = 13
# Keep the first `num_mfccs` MFCCs.
mfccs = tf.contrib.signal.mfccs_from_log_mel_spectrograms(
    log_mel_spectrograms)[..., :num_mfccs]

In [23]:
classifier = tf.estimator.DNNClassifier(
    feature_columns = mfccs,
    hidden_units = [10,10],
    n_classes = 2,
)

RuntimeError: Estimators are not supported when eager execution is enabled.